# Coursera IBM Data Science Capstone

### In this project I will segment a city into different Neighborhoods using the geographical coordinates of the center of each Neighborhood, and then using a combination of location data and machine learning to cluster it.

In [20]:
# importing libaries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

## Segmenting and Clustering Neighberhood

### Scraping Neighborhoods from wiki

In [21]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
# print(soup.prettify())

In [22]:
table_elements = soup.find_all('td')

In [23]:
#building lists with list-comprehension

#add every 3 element to list postcode, starting from element 0
postcode = [table_elements[x].text for x in range(0, len(table_elements), 3)]

#add every 3 element to list borough, starting from element 1
borough = [table_elements[x].text for x in range(1, len(table_elements), 3)]

#add every 3 element to list Neighborhood, starting from element 2
Neighborhood = [table_elements[x].text for x in range(2, len(table_elements), 3)]

In [24]:
#checking the length of the lists
len_post = len(postcode)
len_bor = len(borough)
len_nei = len(Neighborhood)
# print(f'len postcode= {len_post}, len borough= {len_bor}, len neighberhood= {len_nei}')

In [25]:
#deleting unnecessary lines 
del postcode[-12:]
del borough [-11:]
del Neighborhood[-11:]

In [26]:
#making the list neighberhood nicer
neighberhood = [word.replace('\n', '') for word in Neighborhood]

In [27]:
len_post = len(postcode)
len_bor = len(borough)
len_nei = len(Neighborhood)
# print(f'len postcode= {len_post}, len borough= {len_bor}, len Neighborhood= {len_nei}')

### Putting the data into a pandas dataframe

In [28]:
#first we need a dictionary:
data = {'postcode': postcode, 'borough': borough, 'Neighborhood': Neighborhood}
#and heer comes the dataframe :-) :
df = pd.DataFrame(data)
df.tail()

,postcode,borough,Neighborhood
283,M8Z,Etobicoke,Mimico NW\n
284,M8Z,Etobicoke,The Queensway West\n
285,M8Z,Etobicoke,Royal York South West\n
286,M8Z,Etobicoke,South of Bloor\n
287,M9Z,Not assigned,Not assigned\n


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 3 columns):
postcode        288 non-null object
borough         288 non-null object
Neighborhood    288 non-null object
dtypes: object(3)
memory usage: 6.8+ KB


In [30]:
#dropping "not assigned" in borough
df.borough.replace('Not assigned', np.NaN, inplace=True)
df.dropna(inplace=True)
df.head()

,postcode,borough,Neighborhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n


In [31]:
df[df['Neighborhood'].str.contains('Not')]

,postcode,borough,Neighborhood
8,M7A,Queen's Park,Not assigned\n


In [32]:
df.Neighborhood.replace('Not assigned', "Queen's Park", inplace=True)

In [33]:
len(df.postcode.unique())

103

In [34]:
# combining rows with the same postalcde
#into one row with the Neighborhoods separated with a comma
df_grouped = df.groupby(['postcode','borough'])['Neighborhood'].apply(', '.join).reset_index()
df_grouped.head()

,postcode,borough,Neighborhood
0,M1B,Scarborough,"Rouge\n, Malvern\n"
1,M1C,Scarborough,"Highland Creek\n, Rouge Hill\n, Port Union\n"
2,M1E,Scarborough,"Guildwood\n, Morningside\n, West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n


In [35]:
df_grouped.shape

(103, 3)

### Adding Lattitude and Longitude to the Dataframe

In [36]:
# # Test with the suggested methoed from coursera

# import geocoder # import geocoder

# postcodes = df['postcode']

# lat = []
# lon = []

# attempts = 1

# for postcode in postcodes:
#     # initialize your variable to None
#     lat_lng_coords = None

#     # loop until you get the coordinates
#     while(lat_lng_coords is None) and attempts < 5:
#         g = geocoder.google(f'{postcode}, Toronto, Ontario')
#         print('g = ',g)
#         lat_lng_coords = g.latlng
#         print('lat_lng_coords= ', lat_lng_coords)
#         attempts += 1
        

#     latitude = lat_lng_coords[0]
#     longitude = lat_lng_coords[1]
    
#     lat.append(latitude)
#     lon.append(longitude)

# lat

In [37]:
# #Found an other library (https://github.com/symerio/pgeocode) 
# #(I got the results that I wanted, but 
# #I couldn't work with the given datatype(type(nomi)=pgeocode.Nominatim))

# import pgeocode

# df['postcode'] = "5CA " + df['postcode']

# postcode_list = df['postcode'].values.tolist()

# nomi = pgeocode.Nominatim('CA')
# nomi.query_postal_code(postcode_list)


In [38]:
# # # Importing Data from csv file with IBM Cloud
# Deleted because of the api keys


In [39]:
# Importing Data from csv file
geo_data = pd.read_csv('Geospatial_Coordinate.csv')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
#combining the tables
geo_data.rename(columns={'Postal Code': 'postcode'}, inplace=True)
merged_data =pd.merge(df_grouped, geo_data, on='postcode')

In [41]:
merged_data.head()

,postcode,borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge\n, Malvern\n",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek\n, Rouge Hill\n, Port Union\n",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n, Morningside\n, West Hill\n",43.763573,-79.188711
3,M1G,Scarborough,Woburn\n,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476


In [42]:
merged_data.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
postcode        103 non-null object
borough         103 non-null object
Neighborhood    103 non-null object
Latitude        103 non-null float64
Longitude       103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


## Segmenting and Clustering Neighborhoods in Toronto

In [43]:
# importing libaries

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [45]:
# Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# print(f'The geograpical coordinate of Toronto are: latitude= {latitude}, longitude= {longitude}.')

In [46]:
# create a map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighborhood in zip(merged_data['Latitude'], merged_data['Longitude'], merged_data['borough'], merged_data['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [47]:
# @hidden_cell
# Define Foursquare Credentials and Version
CLIENT_ID = # your Foursquare ID
CLIENT_SECRET =  # your Foursquare Secret
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5ZKENUDBF3GUOSFG2RGUDJA0CJMNABWPOWOP3MA5Q25KPD1H
CLIENT_SECRET:NFQUCXAHACFKSAVQ21BLFRKZFWGAGFF0BJJWLJKT20YRUMES


### Explore Neighborhoods in Toronto

In [48]:
# a function to get the top 100 venues for each Neighborhood within a radius of 500 meters

LIMIT = 100 # limit of number of venues returned by Foursquare API

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
# run the above function on each Neighborhood and create a new dataframe

toronto_venues = getNearbyVenues(names=merged_data['Neighborhood'],
                                   latitudes=merged_data['Latitude'],
                                   longitudes=merged_data['Longitude']
                                  )



Rouge
, Malvern

Highland Creek
, Rouge Hill
, Port Union

Guildwood
, Morningside
, West Hill

Woburn

Cedarbrae

Scarborough Village

East Birchmount Park
, Ionview
, Kennedy Park

Clairlea
, Golden Mile
, Oakridge

Cliffcrest
, Cliffside
, Scarborough Village West

Birch Cliff
, Cliffside West

Dorset Park
, Scarborough Town Centre
, Wexford Heights

Maryvale
, Wexford

Agincourt

Clarks Corners
, Sullivan
, Tam O'Shanter

Agincourt North
, L'Amoreaux East
, Milliken
, Steeles East

L'Amoreaux West

Upper Rouge

Hillcrest Village

Fairview
, Henry Farm
, Oriole

Bayview Village

Silver Hills
, York Mills

Newtonbrook
, Willowdale

Willowdale South

York Mills West

Willowdale West

Parkwoods

Don Mills North

Flemingdon Park
, Don Mills South

Bathurst Manor
, Downsview North
, Wilson Heights

Northwood Park
, York University

CFB Toronto
, Downsview East

Downsview West

Downsview Central

Downsview Northwest

Victoria Village

Woodbine Gardens
, Parkview Hill

Woodbine Heights

Th

In [50]:
toronto_venues.shape

(2237, 7)

In [51]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge\n, Malvern\n",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge\n, Malvern\n",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek\n, Rouge Hill\n, Port Union\n",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek\n, Rouge Hill\n, Port Union\n",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood\n, Morningside\n, West Hill\n",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [52]:
toronto_venues.Neighborhood = [word.replace('\n', '') for word in toronto_venues.Neighborhood]

In [53]:
#Let's check how many venues were returned for each Neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",12,12,12,12,12,12
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,56,56,56,56,56,56


In [54]:
#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 278 uniques categories.


### Analyze Each Neighborhood

In [55]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move Neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
toronto_onehot.shape

(2237, 278)

In [57]:
# grouping rows by Neighborhood and by taking the mean of 
# the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [58]:
# each Neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3  American Restaurant  0.04
4           Steakhouse  0.04


----Agincourt----
            venue  freq
0          Lounge  0.25
1  Breakfast Spot  0.25
2    Skating Rink  0.25
3  Clothing Store  0.25
4     Yoga Studio  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0           Playground  0.33
1          Coffee Shop  0.33
2                 Park  0.33
3   Mexican Restaurant  0.00
4  Monument / Landmark  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.17
1             Pharmacy  0.08
2           Beer Store  0.08
3       Sandwich Place  0.08
4  Fried Chicken Joint  0.08


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place   0.2
1            

                       venue  freq
0                Coffee Shop  0.33
1                       Park  0.33
2          Convenience Store  0.33
3                Yoga Studio  0.00
4  Middle Eastern Restaurant  0.00


----Emery, Humberlea----
                             venue  freq
0                   Baseball Field   1.0
1                      Yoga Studio   0.0
2        Middle Eastern Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.12
1  Fast Food Restaurant  0.07
2           Coffee Shop  0.07
3            Kids Store  0.03
4   Japanese Restaurant  0.03


----First Canadian Place, Underground city----
                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.09
2                Hotel  0.04
3  American Restaurant  0.03
4        Deli / Bodega  0.03


----Flemingdon Park, Don Mills South----
              venue  freq
0     

                             venue  freq
0                Health Food Store  0.25
1                              Pub  0.25
2                      Coffee Shop  0.25
3              Monument / Landmark  0.00
4  Molecular Gastronomy Restaurant  0.00


----The Beaches West, India Bazaar----
                  venue  freq
0        Sandwich Place  0.10
1  Fast Food Restaurant  0.05
2            Board Shop  0.05
3      Sushi Restaurant  0.05
4          Burger Joint  0.05


----The Danforth West, Riverdale----
                    venue  freq
0        Greek Restaurant  0.21
1             Coffee Shop  0.10
2      Italian Restaurant  0.07
3          Ice Cream Shop  0.07
4  Furniture / Home Store  0.05


----The Junction North, Runnymede----
                  venue  freq
0  Caribbean Restaurant  0.33
1     Convenience Store  0.33
2              Bus Line  0.33
3           Yoga Studio  0.00
4     Mobile Phone Shop  0.00


----The Kingsway, Montgomery Road, Old Mill North----
                       ven

In [59]:
# putting that data into a pandas df
# first: function to sort the venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:
# creating the new df and displaying the top 5 venues

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    Neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

Neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Bar,Steakhouse
1,Agincourt,Breakfast Spot,Lounge,Skating Rink,Clothing Store,Donut Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Coffee Shop,Women's Store,Doner Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Beer Store,Fried Chicken Joint,Japanese Restaurant,Fast Food Restaurant
4,"Alderwood, Long Branch",Pizza Place,Gym,Pharmacy,Athletics & Sports,Pub


## Clustering Neighborhoods

In [61]:
# Run k-means to cluster the Neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 3, 0, 0, 0])

In [62]:
# create a new dataframe that includes the cluster 
# as well as the top 10 venues for each Neighborhood

# adjust the Neighborhood column in the merged_data dataframe
merged_data.Neighborhood = [word.replace('\n', '') for word in merged_data.Neighborhood]

# add clustering labels
Neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = merged_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each Neighborhood
toronto_merged = toronto_merged.join(Neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(200) # check the last columns!

,postcode,borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0.0,Print Shop,Fast Food Restaurant,Women's Store,Department Store,Dessert Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.0,History Museum,Bar,Women's Store,Donut Shop,Dim Sum Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Electronics Store,Breakfast Spot,Rental Car Location,Mexican Restaurant,Medical Center
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Pharmacy,Korean Restaurant,Doner Restaurant,Department Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4.0,Playground,Women's Store,Doner Restaurant,Department Store,Dessert Shop
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,0.0,Bus Station,Coffee Shop,Hobby Shop,Discount Store,Department Store
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,0.0,Bus Line,Bakery,Soccer Field,Intersection,Bus Station
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,0.0,Motel,American Restaurant,Women's Store,Department Store,Dessert Shop
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0.0,General Entertainment,College Stadium,Café,Skating Rink,Women's Store


## Visualization of the Clusters

In [63]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood']):
#     label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
#         color=rainbow[cluster-1],
        fill=True,
#         fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters